### Copyright-protected material, all rights reserved. (c) University of Vienna.
_Copyright Notice of the corresponding course at Moodle applies. <br> Only to be used in the MRE course._

# MRE Assignment 2 - Digital Audio Processing 

In this assignment you will load, decode, and process digital audio files (e.g., MP3, WAV) using Python. For the following tasks, you will use our suggested libraries (see the setup section). For both audio formats you will extract and process content and some basic metadata. For the following tasks, you will use our suggested libraries (see the setup section). 

In this notebook, you will implement your solution. This notebook will be imported into the "*_def.ipynb" notebook.

Of course you can include code for testing your implementation in this implementation notebook, but code for testing and output generated for testing is not going to be assessed.

Of course, your code for the solutions in this notebook will be inspected and is subject to grading.

## Setup

For general installation instructions, please refer to the ressources given for all the assignments in Moodle.

If the cell below executes without error, you can start the assignment!

In [351]:
# -------- Imports --------
# Please do not change the contents of this cell!

# Imports required by us.
from enum import Enum
import mutagen      # mutagen
from mutagen.mp3 import MP3
from mutagen.id3 import ID3
from mutagen.easyid3 import EasyID3
import wave         # python's built-in wave library
import pandas as pd # pandas
import ffmpeg       # ffmpeg-python wrapper (requires ffmpeg.exe in your system path!)
import subprocess   # for calling local executables such as ffmpeg.exe


In the cells below, place your own imports, global variables, (helper) functions and classes. Feel free to add cells here as you see fit.

In [352]:
# Please place your own imports here.
import os
from IPython.display import Audio

In [353]:
# Place any helper functions, global variables and classes here.

class Criteria(Enum):
    ARTIST = 1
    ALBUM = 2
    GENRE = 3

## Task 2.1 Organize Audio files by specific criteria (35P):

In [354]:
# used following docs:
# https://mutagen.readthedocs.io/en/latest/api/wave.html
# https://mutagen.readthedocs.io/en/latest/user/gettingstarted.html
# https://mutagen.readthedocs.io/en/latest/api/mp3.html

# Auto-plays an audio file and also embeds an IPython audio display.
def MyAudioFilesOrganizer(inputDir: str, grouping) -> pd.DataFrame:
    dataFrame = pd.DataFrame(columns=['filename', 'format', 'encoder', 'duration', 'artist', 'title', 'date', 'album', 'track', 'composer', 'genre', 'sample rate', 'bitrate', 'channels'])
    
    # directory crawler taken from previous Assignment
    if os.path.isdir(inputDir):
        for filename in os.listdir(inputDir):
            filepath = os.path.join(inputDir, filename)
            name, extension = os.path.splitext(filename)
            if os.path.isfile(filepath) and extension.upper() in ['.MP3']:
                
                audio = Audio(filepath, autoplay=False) # autoplay false, as else, all files play at once
                display(audio)
                
                mp3 = MP3(filepath) # reading MP3 and ID3(therein) with mutagen
                
                # printing the MP3, one can see where the details are hidden
                newRow = pd.DataFrame.from_records([{
                    'filename': filename,
                    'format': "MP3",
                    'encoder': mp3["TENC"][0],
                    'duration': mp3.info.length,
                    'artist': mp3["TPE1"][0],
                    'title': mp3["TIT2"][0],
                    'date': mp3["TDRC"][0],
                    'album': mp3["TALB"][0],
                    'track': mp3["TRCK"][0],
                    'composer': mp3["TCOM"][0],
                    'genre': mp3["TCON"][0],
                    'sample rate': mp3.info.sample_rate,
                    'bitrate': mp3.info.bitrate,
                    'channels': mp3.info.channels
                }])
                dataFrame = pd.concat([dataFrame, newRow])
                
                
            if os.path.isfile(filepath) and extension.upper() in ['.WAV']:
                
                audio = Audio(filepath, autoplay=False) # autoplay false, as else, all files play at once
                display(audio)
                
                wave = mutagen.File(filepath) # reading wav with mutagen
                
                # wave cannot carry metadata other than for pcm signal
                newRow = pd.DataFrame.from_records([{
                    'filename': filename,
                    'format': "WAVE",
                    'encoder': "-",
                    'duration': wave.info.length,
                    'artist': "-",
                    'title': "-",
                    'date': "-",
                    'album': "-",
                    'track': "-",
                    'composer': "-",
                    'genre': "-",
                    'sample rate': wave.info.sample_rate,
                    'bitrate': wave.info.bitrate,
                    'channels': wave.info.channels
                }])
                dataFrame = pd.concat([dataFrame, newRow])
                
    # grouping / sorting
    if grouping == Criteria.ARTIST:
        dataFrame = dataFrame.sort_values(by='artist', ascending=False) 
    if grouping == Criteria.ALBUM:
        dataFrame = dataFrame.sort_values(by='album', ascending=False)
    if grouping == Criteria.GENRE:
        dataFrame = dataFrame.sort_values(by='genre', ascending=False)            
    
    dataFrame.reset_index(drop=True, inplace=True)
    return dataFrame

In [355]:
# Test your function here.
#MyAudioFilesOrganizer("./media/audio/", Criteria.ARTIST)

## Task 2.2 Audio mixer (25P):

In [356]:
# References:
# https://kkroening.github.io/ffmpeg-python/
# https://ffmpeg.org/ffmpeg-filters.html
# https://github.com/kkroening/ffmpeg-python/tree/master/examples
# https://stackoverflow.com/questions/65065501/trim-audio-file-using-python-ffmpeg
# https://github.com/kkroening/ffmpeg-python/issues/281#issuecomment-546724993

# Merges two audio files using FFMPEG.
def TwoAudioMixer(audioFile1: str, a1From: int, a1To: int, 
                  audioFile2: str, a2From: int, a2To: int, overlapDur: float, 
                  outputDir: str, outFilename: str) -> None:
    
    # path and dir handling
    outputPath = os.path.join(outputDir, outFilename)
    if not os.path.exists(outputDir):
        os.makedirs(outputDir)

    # reading in the files twice, as ffmpeg gives an error('split it') if file is tried to trim twice (once for non-overlapping part and once for overlapping part)
    audio1Solo = ffmpeg.input(filename=audioFile1).filter(filter_name='atrim', start=a1From, end=a1To-overlapDur)
    audio1Over = ffmpeg.input(filename=audioFile1).filter(filter_name='atrim', start=a1To-overlapDur, end=a1To)
    audio2Over = ffmpeg.input(filename=audioFile2).filter(filter_name='atrim', start=a2From, end=a2From+overlapDur)
    audio2Solo = ffmpeg.input(filename=audioFile2).filter(filter_name='atrim', start=a2From+overlapDur, end=a2To)

    # using mix filter for overlapping part
    overlappedPart = ffmpeg.filter(stream_spec=[audio1Over, audio2Over], filter_name='amix')

    # concatinating all parts sequentially (v=0 for no video stream and a=1 for 1 audio stream)
    finalAudio = ffmpeg.concat(audio1Solo, overlappedPart, audio2Solo, v=0, a=1)

    ffmpeg.output(finalAudio, outputPath).run(overwrite_output=True)


In [357]:
# Test your function here.
#TwoAudioMixer('./media/audio/Amazon.mp3', 0, 6, './media/audio/Hombre.mp3', 6, 12, 2, "output-a2", "t2-mixed.mp3")

## Task 2.3 Concealing speakers ID by lowering/increasing the audio pitch (20P):

In [358]:
# References:
# https://stackoverflow.com/questions/53374590/ffmpeg-change-tone-frequency-keep-length-pitch-audio

# Changes the pitch of an audio file using FFMPEG.
def VoicePitchChanger(audioFile: str, shift: float, outputDir: str, outFilename: str) -> None:
    
    # calculating shift for 'atempo'
    # +12 is one octave up, -12 one octave down
    if shift > 0:
        shift = shift/12 + 1
    if shift < 0:
        shift = 1/((abs(shift/12)+1))
    if shift is 0:
        shift = 1
    
    # path and dir handling
    outputPath = os.path.join(outputDir, outFilename)
    if not os.path.exists(outputDir):
        os.makedirs(outputDir)

    #reading sample-rate with mutagen
    mAudio = mutagen.File(audioFile)
    sr = mAudio.info.sample_rate
    
    audio = ffmpeg.input(filename=audioFile)
    
    # as the tempo inevitably changes when change the pitch (reading sample-rate), we have to artifically strech the signal
    # correcting to the original length (having it after prior to the other operations lead to better results)
    audio = audio.filter(filter_name='atempo', tempo=1/shift) # minimum tempo shift for 'atempo' is 0.5, therefore, a higher shift than 2 in pitch isn't possible, without chaning the speed
                                                                # i tried to use multiple 'atempos' sequentially, but it produced very weird results                              
    # to pitch the signal, we shift - how many samples are read-in per second -> frequencies in the signal shift
    # shifting the sample-rate to the desired pitch, and resample to 'set it in stone'
    audio = audio.filter(filter_name='asetrate', sample_rate=sr*shift)
    audio = audio.filter(filter_name='aresample', sample_rate=sr)
    
    ffmpeg.output(audio, outputPath).run(overwrite_output=True)

<>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
/var/folders/kv/rlm1b_915wsbvd995jlr3f880000gn/T/ipykernel_96495/1990326066.py:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if shift is 0:


In [359]:
# Test your function here.
#VoicePitchChanger('./media/myaudio/Humbum.mp3', 12, "output-a2", "t3-pitched.mp3")
#VoicePitchChanger('output-a2/t3-pitched.mp3', -12, "output-a2", "t3-pitched-back.mp3")